# Semi-supervised and generative models

In the lectures the Variational Autoencoder (VAE) was introduced as a specific generative latent variable model. Compared to probabilistic PCA, VAE employs nonlinear functions (neural network) for encoding and decoding, and its learning is achieved by maximizing the evidence lower bound (ELBO). 


### Exercise 9.1 - Understand and derive the ELBO

Given observed data points $\{ x_n \}_{n=1}^N$ distributed according to some unknown data distribution $p_\mathrm{data}(x)$, the goal of generative models is to learn a model $p_\theta(x)$ that we can sample from, such that $p_\theta(x)$ is as similar to $p_\mathrm{data}(x)$ as possible. 

Let's consider the following latent variable model:

$$
x \sim p_\theta(x|z), \quad z \sim p(z).
$$

where $p(z)$ could be a simple Gaussian and $\theta$ is the parameter of a neural network. Then the marginal likelihood of $x$ can be writen as

$$
p_\theta(x) = \int p_\theta(x | z) p(z) dz,
$$

which is the objective we want to maximize. But unfortunately, this integral is usually intractable due to the nonlinar and high-dimensional functions (i.e., neural network) inside $z$! Instead, we try to find a lower bound on $p_\theta(x)$ that is easier to compute, which gives the ELBO objective.

Specifically, we focus on maximizing the log-likelihood $\log p_\theta(x)$ by introducing an approximate posterior $q_\phi(z|x)$, as

$$
\log p_\theta(x) = \log \int p_\theta(x|z) p(z) \, dz = \log \int q_\phi(z|x)\,\frac{p_\theta(x|z) p(z)}{q_\phi(z|x)} dz = \log \mathbb{E}_{q_\phi(z|x)}\left[\frac{p_\theta(x|z) p(z)}{q_\phi(z|x)}\right].
$$

Applying [Jensen's Inequality](https://en.wikipedia.org/wiki/Jensen%27s_inequality) to this equation results in the **E**vidence **L**ower **BO**und (ELBO):

$$
\begin{aligned}
\mathcal{L}(x; \theta, \phi) &= \log \mathbb{E}_{q_\phi(z|x)}\left[\frac{p_\theta(x|z) p(z)}{q_\phi(z|x)}\right] \\ 
&\geq \mathbb{E}_{q_\phi(z|x)}\left[\log \frac{p_\theta(x|z) p(z)}{q_\phi(z|x)}\right] \\ 
&= \mathbb{E}_{q_\phi(z|x)}\left[\log p_\theta(x|z)\right] - D_{KL}(q_\phi(z|x) \| p(z)).
\end{aligned}
$$

The first term $\mathbb{E}_{q_\phi(z|x)}\left[\log p_\theta(x|z)\right]$ is called the reconstruction loss maximizing the log-likelihood and the second term $-D_{KL}(q_\phi(z|x) \| p(z))$ is a regularizer minimizing the distance between $q_\phi(z|x)$ and $p(z)$.

So, optimizing the ELBO means learning an encoder ($\phi$) and a decoder ($\theta$) that fits the dataset while ensuring the encoded latent variable $z$ remains close to the prior $p(z)$.

### Exercise 9.2 - Implement the VAE

In this exercise, you will learn the key techniques that make VAEs actually trainable and will implement the VAE on a small dataset.

#### Compute the KL divergence

Note that the KL term has a closed-form if both the approximate posterior $q_\phi(z|x)$ and prior $p(z)$ are Gaussian distribution. For example, we define $p(z)$ a *standard multivariate Gaussian* and $q_\phi(z|x)$ a multivariate Gaussian with a diagonal covariance matrix:

$$
p(z) = \mathcal{N}(0, I), \quad q_\phi(z|x) = \mathcal{N}(\mu_\phi(x), \mathrm{diag}(\sigma^2_\phi(x))).
$$

The [KL divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence#Examples) between them has a simple yet analytical solution given by

$$
D_{KL}(q_\phi(z|x) \| p(z)) = -\frac{1}{2} \sum_{n=1}^N (1 + \log (\sigma_\phi^2(x_n)) - \mu_\phi^2(x_n) - \sigma_\phi^2(x_n)),
$$

where $N$ is the number of dimensions in the latent space. $\mu_\phi$ and $\sigma_\phi^2$ are the mean and variance generated by the encoder, respectively.

**(a) KL Implementation:**

#### Reparameterization trick 

The question now turns to how to compute the expectation over posterior i.e., $\mathbb{E}_{z \sim q_\phi(z|x)}\left[\log p_\theta(x|z)\right]$). However, the latent variable $z$ is sampled from $q_\phi(z | x)$, whose parameter $\phi$ is what we want to optimize as well. And the sampling process itself, such as , is not differentiable. Then how can we compute the gradient $\nabla_\phi$ from the random sampling node and make sure it is avaiable in backpropagation? The solution is the *Reparameterization Trick*.

The trick is to separate the randomness from the parameters. Instead of directly sampling $z \sim q_\phi(z | x)$, we generate $z$ from the following two steps:

1. Sample a random noise from the standard normal distribution: $\epsilon \sim \mathcal{N}(0, I)$.
2. Obtain $z$ from a deterministic, differentiable function: $z = \mu_\phi(x) + \sigma_\phi(x) \cdot \epsilon$.

In this way, $z$ is still a random variable with distribution $\mathcal{N}(\mu_\phi(x), \sigma^2_\phi(x))$, but the randomness comes entirely from $\epsilon$ rather than the encoder.

**(b) Sampling Implementation:**

**(c) The final ELBO loss function:**

Now we can define the Network architecture and train the VAE model.

**Task:** Read through and understand the implementation of the training procedure.

### Exercise 9.3 - Model Inference

Once trained, we can obtain new data points by sampling from the noise distribution.